In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import polars as pl
import numpy as np
# get data path from project directory
from pathlib import Path
import os
from os.path import join, split
import sys
from tqdm import tqdm
data_path = join(Path(os.getcwd()).parent.parent, 'data')
mimic_path = join(data_path, 'raw', 'mimic-iii-clinical-database-1.4')
extr_path = join(data_path, 'interim', 'mimic_iii_ml_for_health')
import dask.dataframe as dd
import gzip, shutil

# We will only process tables with data not contained in the mimic extractor

In [19]:
dfi = pd.read_hdf(join(extr_path, "all_hourly_data.h5"), 
    key='interventions')
dfi

vent  vaso  adenosine  dobutamine  \
subject_id hadm_id icustay_id hours_in                                      
3          145834  211552     0            1     0          0           0   
                              1            1     1          0           0   
                              2            1     1          0           0   
                              3            1     1          0           0   
                              4            1     1          0           0   
...                                      ...   ...        ...         ...   
99999      113369  246512     22           0     0          0           0   
                              23           0     0          0           0   
                              24           0     0          0           0   
                              25           0     0          0           0   
                              26           0     0          0           0   

                                        dopamine  epinephrine  isuprel  \
subject_id hadm_id icustay_id hours_in                                   
3          145834  211552     0                0            0        0   
                              1                1            0        0   
                              2                1            0        0   
                              3                0            0        0   
                              4                0            0        0   
...                                          ...          ...      ...   
99999      113369  246512     22               0            0        0   
                              23               0            0        0   
                              24               0            0        0   
                              25               0            0        0   
                              26               0            0        0   

                                        milrinone  norepinephrine  \
subject_id hadm_id icustay_id hours_in                              
3          145834  211552     0                 0               0   
                              1                 0               0   
                              2                 0               0   
                              3                 0               0   
                              4                 0               1   
...                                           ...             ...   
99999      113369  246512     22                0               0   
                              23                0               0   
                              24                0               0   
                              25                0               0   
                              26                0               0   

                                        phenylephrine  vasopressin  \
subject_id hadm_id icustay_id hours_in                               
3          145834  211552     0                     0            0   
                              1                     1            0   
                              2                     1            0   
                              3                     1            0   
                              4                     1            0   
...                                               ...          ...   
99999      113369  246512     22                    0            0   
                              23                    0            0   
                              24                    0            0   
                              25                    0            0   
                              26                    0            0   

                                        colloid_bolus  crystalloid_bolus  \
subject_id hadm_id icustay_id hours_in                                     
3          145834  211552     0                     0                  0   
                              1       

In [18]:
dfvl = pd.read_hdf(join(extr_path, "all_hourly_data.h5"), 
    key='vitals_labs_mean')
dfvl

LEVEL2                                 alanine aminotransferase albumin  \
Aggregation Function                                       mean    mean   
subject_id hadm_id icustay_id hours_in                                    
3          145834  211552     0                            25.0     1.8   
                              1                             NaN     NaN   
                              2                             NaN     NaN   
                              3                             NaN     NaN   
                              4                             NaN     NaN   
...                                                         ...     ...   
99999      113369  246512     22                            NaN     NaN   
                              23                            NaN     NaN   
                              24                            NaN     NaN   
                              25                            NaN     NaN   
                              26                            NaN     NaN   

LEVEL2                                 albumin ascites albumin pleural  \
Aggregation Function                              mean            mean   
subject_id hadm_id icustay_id hours_in                                   
3          145834  211552     0                    NaN             NaN   
                              1                    NaN             NaN   
                              2                    NaN             NaN   
                              3                    NaN             NaN   
                              4                    NaN             NaN   
...                                                ...             ...   
99999      113369  246512     22                   NaN             NaN   
                              23                   NaN             NaN   
                              24                   NaN             NaN   
                              25                   NaN             NaN   
                              26                   NaN             NaN   

LEVEL2                                 albumin urine alkaline phosphate  \
Aggregation Function                            mean               mean   
subject_id hadm_id icustay_id hours_in                                    
3          145834  211552     0                  NaN               73.0   
                              1                  NaN                NaN   
                              2                  NaN                NaN   
                              3                  NaN                NaN   
                              4                  NaN                NaN   
...                                              ...                ...   
99999      113369  246512     22                 NaN                NaN   
                              23                 NaN                NaN   
                              24                 NaN                NaN   
                              25                 NaN                NaN   
                              26                 NaN                NaN   

LEVEL2                                  anion gap asparate aminotransferase  \
Aggregation Function                         mean                      mean   
subject_id hadm_id icustay_id hours_in                                        
3          145834  211552     0         20.666667                      69.0   
                              1               NaN                       NaN   
                              2               NaN                       NaN   
                              3               NaN                       NaN   
                              4               NaN                       NaN   
...                                           ...                       ...   
99999      113369  246512     22              NaN                       NaN   
                              23              NaN                      

In [17]:
dfp = pd.read_hdf(join(extr_path, "all_hourly_data.h5"),
    key='patients')
dfp

,,,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,discharge_location,fullcode_first,dnr_first,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,SNF,1.0,0.0,...,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0,0,0,1,0,145
4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,HOME WITH HOME IV PROVIDR,1.0,0.0,...,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0,0,0,1,0,40
6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,HOME HEALTH CARE,1.0,0.0,...,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0,0,0,1,0,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,DEAD/EXPIRED,1.0,0.0,...,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1,1,1,1,0,127
11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,HOME HEALTH CARE,1.0,0.0,...,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0,0,0,1,0,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99983,117390,286606,M,UNKNOWN/NOT SPECIFIED,78.576624,Medicare,2193-04-26 11:35:00,ST ELEVATION MYOCARDIAL INFARCTION;CORONARY AR...,2193-04-29 13:30:00,HOME,NaN,NaN,...,2193-04-27 12:33:22,1.039942,EMERGENCY,CCU,0,0,0,1,0,24
99991,151118,226241,M,WHITE,47.729259,Private,2184-12-24 08:30:00,DIVERTICULITIS/SDA,2185-01-05 12:15:00,HOME,1.0,0.0,...,2184-12-31 20:56:20,3.142616,ELECTIVE,TSICU,0,0,0,1,0,75
99992,197084,242052,F,WHITE,65.772155,Medicare,2144-07-25 18:03:00,RETROPERITONEAL HEMORRHAGE,2144-07-28 17:56:00,SNF,1.0,0.0,...,2144-07-27 17:27:55,1.974456,EMERGENCY,MICU,0,0,0,1,0,47


## ICD codes

In [22]:
df_diag = pd.read_csv(join(mimic_path, 'DIAGNOSES_ICD.csv.gz'),  compression='gzip')
df_diag

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


We need to construct a mapping from ICD10 to ICD9 codes if we want to use pretrained models on this.

In [24]:
df_diag_d = pd.read_csv(join(mimic_path, 'D_ICD_DIAGNOSES.csv.gz'),  compression='gzip')
df_diag_d

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."
...,...,...,...,...
14562,14432,V7399,Scrn unspcf viral dis,Special screening examination for unspecified ...
14563,14433,V740,Screening for cholera,Screening examination for cholera
14564,14434,V741,Screening-pulmonary TB,Screening examination for pulmonary tuberculosis
14565,14435,V742,Screening for leprosy,Screening examination for leprosy (Hansen's di...


## DRG codes

In [23]:
df_drg = pd.read_csv(join(mimic_path, 'DRGCODES.csv.gz'),  compression='gzip')
df_drg

,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,342,2491,144486,HCFA,28,"TRAUMATIC STUPOR & COMA, COMA <1 HR AGE >17 WI...",NaN,NaN
1,343,24958,162910,HCFA,110,MAJOR CARDIOVASCULAR PROCEDURES WITH COMPLICAT...,NaN,NaN
2,344,18325,153751,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN
3,345,17887,182692,HCFA,14,SPECIFIC CEREBROVASCULAR DISORDERS EXCEPT TRAN...,NaN,NaN
4,346,11113,157980,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN
...,...,...,...,...,...,...,...,...
125552,123452,71582,101422,MS,221,CARDIAC VALVE & OTH MAJ CARDIOTHORACIC PROC W/...,NaN,NaN
125553,123453,46449,110075,APR,1653,Coronary Bypass w/ Cardiac Cath Or Percutaneou...,3.0,2.0
125554,123454,46449,110075,APR,1653,Coronary Bypass w/ Cardiac Cath Or Percutaneou...,3.0,2.0
125555,123455,46449,110075,MS,234,CORONARY BYPASS W CARDIAC CATH W/O MCC,NaN,NaN


Maybe relevant, check overlap with ICD codes

## Procedures

In [25]:
df_pro = pd.read_csv(join(mimic_path, 'PROCEDURES_ICD.csv.gz'),  compression='gzip')
df_pro

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331
...,...,...,...,...,...
240090,228330,67415,150871,5,3736
240091,228331,67415,150871,6,3893
240092,228332,67415,150871,7,8872
240093,228333,67415,150871,8,3893


Time of the procedures not specified

In [ ]:
df_ce = pd.read_csv(join(mimic_path, 'CHARTEVENTS.csv.gz'), compression='gzip', nrows=50000)
print('RESULTSTATUS',df_ce.RESULTSTATUS.unique(), 'STOPPED', df_ce.STOPPED.unique(), 'WARNING',df_ce.WARNING.unique())
print('STORETIME can be dropped, because its the time of entering the data into the database, CHARTTIME is the time of the measurement')
print('CGID can be dropped as we don\'t care about the care giver')
print('VALUENUM can be dropped, because we have the VALUE column which containes the full information')
print('We can mask out on the ERROR column and drop the ERROR column')
print('ROW_ID does not carry any information')

In [ ]:
all_columns = pd.read_csv(join(mimic_path, 'CHARTEVENTS.csv.gz'), compression='gzip', nrows=1).columns
drop_columns = ['RESULTSTATUS', 'STOPPED','WARNING', 'STORETIME', 'CGID', 'VALUENUM', 'ROW_ID']
load_columns = [c for c in all_columns if not c in drop_columns]
df_ce = pd.read_csv(join(mimic_path, 'CHARTEVENTS.csv.gz'), compression='gzip', nrows=int(1e6), usecols=load_columns, parse_dates=['CHARTTIME'])
# df_ce = df_ce[df_ce['ERROR'] == 0]
# df_ce = df_ce.drop(columns=['ERROR'])
print(sys.getsizeof(df_ce)/1e6, 'MB')

In [ ]:
def load_csv(path, columns_str=None, columns=None):
    if path.endswith('.gz'):
        compression = 'gzip'
    else:
        compression = None
    if not columns_str is None:
        all_columns = pl.read_csv(path, n_rows=1, low_memory=True).columns
        selected_columns = [c for c in all_columns for s in columns_str if columns_str in c]
        if not columns is None:
            columns = list(set(columns + selected_columns))
        else:
            columns = selected_columns
    df = pl.read_csv(path, columns=columns, low_memory=True, parse_dates=True)
    return df

In [ ]:
def pandas_get_columns(path):
    if path.endswith('.gz'):
        compression = 'gzip'
    return pd.read_csv(path, nrows=1, compression=compression).columns

In [ ]:
def get_dest_path_for_parquet(dest_dir, file_path):
    file_name = split(file_path)[1]
    if not dest_dir is None:
        dest_path = join(dest_dir, file_name.replace('.csv.gz', '.parquet.gz'))
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)
    else:
        file_name = file_name.replace('.csv.gz', '.parquet.gz')
        mimic_dir = split(file_path)[0]
        mimic_dir_name = split(mimic_dir)[1]
        data_dir = split(split(mimic_dir)[0])[0]
        interim_mimic_dir = join(data_dir, 'interim', mimic_dir_name)
        if not os.path.exists(interim_mimic_dir):
            os.makedirs(interim_mimic_dir)
        dest_path = join(interim_mimic_dir, file_name)
    return dest_path

def convert_csv_to_parquet(mimic_path, file_name, dest_dir=None, test=False, nrows=None):
    """
        test: load only 1000 rows
        file_name: path of the csv file, with ending
        dest_dir: directory to store the parquet files, without ending
    """
    mimic_path = mimic_path# cfg.raw_data_path
    dest_dir = dest_dir # cfg.data_working_path
    file_path = join(mimic_path, file_name)
    columns = pandas_get_columns(file_path)
    dtype_dic = {column:"Int64" for column in columns if column.endswith('ID') and column!='FLUID'}
    if test:
        nrows = 300
    dest_path = get_dest_path_for_parquet(dest_dir, file_path)
    pd.read_csv(file_path, nrows=nrows, dtype=dtype_dic, parse_dates=True, compression='gzip').to_parquet(dest_path, compression='gzip', index=False,)


_CV and _MV endings indicate the system used to record the data.\
D prefix is a dictionary table and provides definitions for clinical identifiers.
5 tables to track patients: Admissions, Patients, ICUstays, Services, Transfers

In [ ]:
# group dfs
tracking_table_names = ['ADMISSIONS', 'PATIENTS', 'ICDUSTAYS', 'SERVICES', 'TRANSFERS']
tracking_sheet = [file for file in files for table in tracking_table_names if file.startswith(table)]
events_table_names = ['CALLOUT', 'CAREGIVERS', 'CHARTEVENTS', 'CPTEVENTS', 'DATETIMEEVENTS', 'DIAGNOSES_ICD', 'DRGCODES', 'INPUTEVENTS', 'LABEVENTS', 'NOTEEVENTS', 'OUTPUTEVENTS', 'PRESCRIPTIONS', 'PROCEDUREEVENTS', 'PROCEDURES_ICD']
events_sheet = [file for file in files for table in events_table_names if file.startswith(table)]
dictionary_sheet = [table for table in files if table.startswith('D_')]

In [ ]:
events_sheet[0].strip('.csv.gz')

In [ ]:
tables = tracking_sheet + events_sheet + dictionary_sheet

In [ ]:
df_dt = pd.read_csv(join(mimic_path, 'DATETIMEEVENTS.csv.gz'), nrows=100000, compression='gzip')
df_dt


In [ ]:
df_dt.VALUEUOM.unique()

In [ ]:
load_csv(join(mimic_path, 'D_LABITEMS.csv.gz'))

In [ ]:
df_adm = pd.read_csv(join(mimic_path, tracking_sheet[0]), compression='gzip')
df_adm

In [ ]:
df_adm = pd.read_csv(join(mimic_path, 'ADMISSIONS.csv.gz'), compression='gzip')
print('length', len(df_adm))
print('unique patients', len(df_adm['SUBJECT_ID'].unique()))
df_adm.head()

In [ ]:
df_co = pd.read_csv(join(mimic_path, 'CALLOUT.csv.gz'), compression='gzip')
df_co.head()
print('length', len(df_co))
print('unique patients', len(df_co['SUBJECT_ID'].unique()))
df_co.head()

In [ ]:
df_co = ps.read_csv(join(mimic_path, 'CALLOUT.csv.gz'))
sys.getsizeof(df_co)

In [ ]:
# not relevant for our analysis
df_cg = pd.read_csv(join(mimic_path, 'CAREGIVERS.csv.gz'), compression='gzip')

In [ ]:
pd_df = pd.read_csv(join(mimic_path, 'CHARTEVENTS.csv.gz'), compression='gzip', nrows=50000)
#pd_df.to_parquet(join(mimic_path, 'CHARTEVENTS_sample.parquet'), index=False)

In [ ]:
df_ps = pl.read_csv(join(mimic_path, 'CHARTEVENTS.csv.gz'),  n_rows=5000, parse_dates=True)

In [ ]:
# no information on type of event, only value and ID
df_ce = pd.read_csv(join(mimic_path, 'CHARTEVENTS.csv.gz'), compression='gzip', nrows=10000)
df_ce.head()

General approach:
- createa dataframe which contains: event_name, timestamp, value, visit, age 
- separate dataframes into batches